Idee: Start und Endzustand des Springenden Atoms umbenennen und so tun als wäre das der Sprungzustande

Todo:   - springendes Atom Identifizieren - check
        - neue Daten ausschreiben

In [1]:
import glob
import os


In [14]:

import ase
from ase.io import read as ASEread
import numpy as np

def identify_jumping_atom(atoms_init, atoms_final):

    # apply minimum image convention to get the displacement vector in direct coordinates for every atom
    init_pos  = atoms_init.get_scaled_positions()
    final_pos = atoms_final.get_scaled_positions()
    for i, (pos_init, pos_fin) in enumerate(zip(init_pos, final_pos)):
        for xyz in [0,1,2]:
            if pos_fin[xyz] - pos_init[xyz] > 0.5:
                final_pos[i][xyz] = final_pos[i][xyz] - 1
            elif pos_fin[xyz] - pos_init[xyz] < -0.5:
                final_pos[i][xyz] = final_pos[i][xyz] + 1
    rel_displacement_vector = final_pos - init_pos
    
    # get cell and compute the displacement vector in cartesian form
    (a,b,c) = atoms_init.get_cell()
    displacement_vector = []
    for (x, y, z) in rel_displacement_vector:
        displacement_vector.append(x*a+y*b+z*c)
    displacement_vector = np.array(displacement_vector)
    
    # get displacement magnitude and check for the ones with displacement larger than 2
    # which should be just the Li that jumps. Get its index
    displacement = np.sqrt(np.sum(displacement_vector ** 2, axis = 1))
    count = 0
    for i, displ in enumerate(displacement):
        if displ > 2:
            count += 1
            Li_index = i
    
    # Check that there are not >1 atoms moving 
    if count != 1:
        raise Exception(f"WARNING: There are {count} ions that move more than 2 Ang during the NEB! Check NEB path again!")
    else:
        return Li_index

In [6]:
# Define the source and destination directories
source_base = "/nfshome/sadowski/work/LiNiO2_data_base_Sabrina/DFT_database/NEBs_Marcel"
destination_base = "/nfshome/winkelmann/ARL/NEBs_Marcel"

#find all folders with final data and copy them, so I got write access for creating the structures
# the ones created manually (0250, 0500, 0750)
folders = glob.glob(f"{source_base}/*/NEB*/run_final*")
# the random ones
folders += glob.glob(f"{source_base}/*random*/02_odh/NEB*/run_final*")
folders += glob.glob(f"{source_base}/*random*/03_tsh/NEB*/run_final*")
folders += glob.glob(f"{source_base}/*random*/04_double_tsh/NEB*/run_final*")

## kopieren von Marcels daten

In [16]:
# Copy each folder to the destination, maintaining the folder structure
for folder in folders:
    # Create the destination path by replacing the source base with the destination base
    source_folder = '/'.join(folder.split('/')[0:-1])
    destination_folder = source_folder.replace(source_base, destination_base)

    if os.path.exists(destination_folder):
        print(f"Folder {destination_folder} already exists. Skipping...")
        continue

    # Create the destination directory if it doesn't exist
    os.makedirs(destination_folder, exist_ok=True)

    # copy the run_final* folder
    os.system(f"cp -r {folder} {destination_folder}")

    # Copy the OUTCAR_final_image and OUTCAR_initial_image files
    source_folder = '/'.join(folder.split('/')[0:-1])
    outcar_initial = source_folder + '/OUTCAR_initial_image'
    outcar_final = source_folder + '/OUTCAR_final_image'
    os.system(f"cp {outcar_final} {destination_folder}")
    os.system(f"cp {outcar_initial} {destination_folder}")


Folder /nfshome/winkelmann/ARL/NEBs_Marcel/0750/NEB_04-15_finished already exists. Skipping...
Folder /nfshome/winkelmann/ARL/NEBs_Marcel/0750/NEB_05-16_finished already exists. Skipping...
Folder /nfshome/winkelmann/ARL/NEBs_Marcel/0750/NEB_06-17_finished already exists. Skipping...
Folder /nfshome/winkelmann/ARL/NEBs_Marcel/0750/NEB_07-18_finished already exists. Skipping...
Folder /nfshome/winkelmann/ARL/NEBs_Marcel/0750/NEB_09-20_finished already exists. Skipping...
Folder /nfshome/winkelmann/ARL/NEBs_Marcel/0750/NEB_10-21_finished already exists. Skipping...
Folder /nfshome/winkelmann/ARL/NEBs_Marcel/0750/NEB_11-22_finished already exists. Skipping...
Folder /nfshome/winkelmann/ARL/NEBs_Marcel/0750/NEB_12-23_finished already exists. Skipping...
Folder /nfshome/winkelmann/ARL/NEBs_Marcel/0750/NEB_13-24_finished already exists. Skipping...
Folder /nfshome/winkelmann/ARL/NEBs_Marcel/0750/NEB_16-17_finished already exists. Skipping...
Folder /nfshome/winkelmann/ARL/NEBs_Marcel/0750/NE

## Aus NEB-folder Struktur erstellen
Schritt 00 und 06 einlesen jumping Li-umbenennen in Struktur 00 und position des jumping Li aus 06 hinzufügen

In [17]:
#Li-trans
atoms_for_training_NEB_transition_states = []
H_o_M_for_training_NEB_transition_states = []
paths_for_training_NEB_transition_states = []

# Iterate over all run_final folders
for folder in folders:

    try:
        #switching the path to the copied files
        source_folder = '/'.join(folder.split('/')[0:-1])
        destination_folder = source_folder.replace(source_base, destination_base)

        # Check the energy along the path. Use initial and final energies from the corresponding relaxed structures + the last steps of the
        # optimized intermediate images
        start_structure = ASEread(destination_folder + '/OUTCAR_initial_image')
        end_structure = ASEread(destination_folder + '/OUTCAR_final_image')
        i = identify_jumping_atom(start_structure, end_structure)
        write_structure = start_structure.copy()
        write_structure[i].symbol = 'Ti'
        end_structure[i].symbol = 'Ti'
        write_structure.append(end_structure[i])
        write_structure.write(destination_folder + '/anchor_trans_image.vasp', format='vasp')
    except Exception as error:
        print(f'{destination_folder} \t {error} \n')
    
    


/nfshome/winkelmann/ARL/NEBs_Marcel/0500/NEB_07-16_finished 	 WARNING: There are 168 ions that move more than 2 Ang during the NEB! Check NEB path again! 

